#### Turn an image into a Feature Vector using Inception
https://tfhub.dev/google/imagenet/inception_v3/feature_vector/3

In [1]:
import tensorflow as tf

from getConfig import getConfig
from mapImageToClass import mapImageToClass
from formatData import formatData
from createDataset import createDataset

In [3]:
# Get the Feature Vector (bottleneck) files for the images; map them to a Class, e.g. "cat"
config = getConfig()
imageClass, classIdx = mapImageToClass(config)
# This process only works on the .npy so remove anything else
for d in list(imageClass):
    if ".npy" not in d:
        imageClass.pop(d)

In [4]:
numClasses = len(classIdx)

In [5]:
dataDict = formatData(config, imageClass, classIdx)

In [6]:
tf.reset_default_graph()

In [7]:
epochs = 100
with tf.name_scope("inputPipeline"):
    trainDS = createDataset(dataDict, epochs, config, "train")
    valDS = createDataset(dataDict, _, config, "val")
    
    iter = tf.data.Iterator.from_structure(trainDS.output_types, tf.compat.v1.data.get_output_shapes(trainDS))
    features, labels = iter.get_next()
    
    trainInit = iter.make_initializer(trainDS)
    valInit = iter.make_initializer(valDS)

W0824 17:23:10.609579 140560228796160 deprecation.py:323] From <ipython-input-7-c96593fc432b>:6: DatasetV1.output_types (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
W0824 17:23:10.616796 140560228796160 deprecation.py:323] From /home/tbrownex/tensorflow/lib/python3.5/site-packages/tensorflow/python/data/ops/iterator_ops.py:348: Iterator.output_types (from tensorflow.python.data.ops.iterator_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
W0824 17:23:10.617967 140560228796160 deprecation.py:323] From /home/tbrownex/tensorflow/lib/python3.5/site-packages/tensorflow/python/data/ops/iterator_ops.py:349: Iterator.output_shapes (from tensorflow.python.data.ops.iterator_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v

In [8]:
L1 = numClasses
STD = 1e-2
LR = 5e-4
bottleneckSize = 2048    # size of the Inception feature vector/bottleneck

with tf.name_scope("Layer1"):
    l1w     = tf.Variable(tf.truncated_normal([bottleneckSize, L1], stddev=STD, dtype=tf.float32))
    l1b     = tf.Variable(tf.truncated_normal([1,L1], dtype=tf.float32))
    output = tf.matmul(features,l1w) + l1b

with tf.name_scope("Accuracy"):
    preds = tf.math.argmax(output, axis=1)
    actuals = tf.math.argmax(labels, axis=1)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(preds, actuals), tf.float32))

xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=output)
loss = tf.reduce_mean(xentropy)
optimize = tf.train.AdamOptimizer(learning_rate=LR).minimize(loss)

W0824 17:23:10.675489 140560228796160 deprecation.py:323] From <ipython-input-8-201c0ba903d1>:16: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
trainBatches = int(len(dataDict["trainX"])/config["batchSize"])
print("Training:    {} batches of size {}".format(trainBatches, config["batchSize"]))
valBatches = int(len(dataDict["valX"])/config["batchSize"])
print("Validation: {} batches of size {}".format(valBatches, config["batchSize"]))

Training:    36 batches of size 32
Validation: 6 batches of size 32


In [11]:
epochs = 30
print("{:<8}{:<8}{}".format("Epoch", "Loss", "Accuracy"))
with tf.Session() as sess:
    writer = tf.summary.FileWriter(config["TBdir"], sess.graph)
    count=0
    sess.run(tf.global_variables_initializer())
    for e in range(epochs):
        sess.run(trainInit)
        for _ in range(trainBatches):
            sess.run(optimize)
        # compute Loss against Val each epoch
        sess.run(valInit)
        totalLoss = 0
        accList = []
        for _ in range(valBatches):
            l, acc = sess.run([loss, accuracy])
            totalLoss += l
            accList.append(acc)
        print("{:<10}{:<10.2f}{:.2%}".format(e+1, totalLoss/valBatches, np.array(accList).mean()))
        sess.run(trainInit)
    writer.close()

Epoch   Loss    Accuracy
1         0.75      81.77%
2         0.56      84.90%
3         0.48      85.94%
4         0.44      85.42%
5         0.39      87.50%
6         0.38      87.50%
7         0.37      86.46%
8         0.35      88.02%
9         0.33      90.10%
10        0.32      90.10%
11        0.34      89.58%
12        0.32      90.62%
13        0.31      90.62%
14        0.32      90.62%
15        0.29      92.19%
16        0.29      90.62%
17        0.30      92.71%
18        0.28      91.67%
19        0.28      91.67%
20        0.27      91.67%
21        0.27      91.15%
22        0.28      90.62%
23        0.28      92.19%
24        0.27      91.67%
25        0.27      91.67%
26        0.28      91.67%
27        0.26      91.67%
28        0.26      91.67%
29        0.26      91.67%
30        0.26      91.67%
